In [1]:

from bs4 import BeautifulSoup 
from urllib.request import urlopen

In [2]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=143437'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html, 'html.parser')

### iframe의 src주소값을 얻어야 함

In [3]:
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=143437&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [4]:
url = url_base + ifr_url
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

### 읽을 페이지 수 계산

In [5]:
count = soup.find('strong', 'total')
count

<strong class="total">관람객 평점 <em>472</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [6]:
count = count.find('em').string     
count

'472'

In [7]:
count = int(count.replace(',',''))
count

472

In [8]:
import math
total_page = math.ceil(count/10)
total_page

48

### 전체 리뷰에서 데이터 수집하기

In [9]:
point = []
review = []
date = []
for page in range(1, total_page+1):
    print(page, end=' ')
    url = url_base + ifr_url + '&page=' + str(page)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    for li in lis:
        score = li.find(class_='star_score').find('em').get_text()
        score = int(score)
        point.append(score)
        count_span = len(li.find(class_='score_reple').find(class_='').find_all('span'))
        count_icon = len(li.find_all(class_='ico_viewer'))
        count_spo = len(li.find_all(class_='text_spo'))
        criteria = str(count_span) + str(count_icon) + str(count_spo)
        #print(criteria)
        if criteria == '100':
            r = li.find(class_='score_reple').find('span').string.strip(' \t\r\n')
        elif criteria == '200':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '210':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')
        elif criteria == '310':
            r = li.find(class_='score_reple').find('a')['data-src']
        elif criteria == '311':
            r = li.find(class_='score_reple').find_all('span')[2].string.strip(' \t\r\n')
        elif criteria == '201':
            r = li.find(class_='score_reple').find_all('span')[1].string.strip(' \t\r\n')

        review.append(r)
        date.append(li.find(class_='score_reple').find_all('em')[1].get_text())

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 

In [10]:
import pandas as pd
df = pd.DataFrame({'평점':point, '일시':date, '감상평':review})
df.tail(10)

,평점,일시,감상평
462,1,2017.03.19 20:39,야 너무한거 아니냐 내가 30년동안 살면서 영화를 다 안보고 나간적은 처음이다 진짜...
463,8,2017.03.27 23:49,아니 그래서 애아빠가 누군데??? 내가 어디 장면을 놓쳤나???
464,7,2019.01.24 03:11,소재는 좋은데 그걸 다 살리지 못함 개연성은 뭐 그냥 틴에이지 물이니까 봐줄만한데 ...
465,10,2018.03.16 02:19,스토리 ost 소재 분위기 다 마음에 들었음오랜만에 미국감성과 연애세포가 살아나는 느낌
466,10,2018.03.16 01:58,그런데 진짜 거지같은 평론가는 왜 존재하는거지 저 찌끄레기들이 8점이상 주는걸 못...
467,7,2019.01.21 10:25,게리 올드만이 아빠입니다
468,10,2018.03.16 02:11,여주 머리묶지말길 머리풀면 이쁜데 머리만 묶으면 이모가 되버려서 남주랑 케미깨짐
469,6,2018.03.02 23:55,주인공 얼굴빨. 아버지가 안 죽어서 유감스럽다.
470,1,2017.03.16 13:13,7샬 연상의 여주 미스케스팅으르 인해 몰입감이...
471,1,2018.04.28 07:51,멍청한 여자들 보라고 만든 하품 나오고 머저리같은 영화


In [11]:
df.to_csv('the_space_between_us.csv')

In [12]:
len(df)

472